In [1]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
import seaborn as sns
from csv import reader
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import apriori, association_rules
pd.set_option("display.max_columns", None)

Leggiamo il dataset

In [2]:
df = pd.read_csv("Groceries_dataset.csv")

Dopo aver letto il dataset prepariamo il dataframe delle transazioni in un formato che permette di poter applicare l'algoritmo Apriori.

Usiamo MLXtend : inseriamo l'oggetto nell'elenco e lo convertiamo
in un df pandas ( Articolo acq = 1 , Articolo non acq = 0 )

In [3]:
DataFrame = pd.DataFrame({'itemDescription' : df.groupby( [ "Member_number", "Date"] )["itemDescription"].apply(list)}).reset_index()
DataFrame

,Member_number,Date,itemDescription
0,1000,15-03-2015,"[sausage, whole milk, semi-finished bread, yog..."
1,1000,24-06-2014,"[whole milk, pastry, salty snack]"
2,1000,24-07-2015,"[canned beer, misc. beverages]"
3,1000,25-11-2015,"[sausage, hygiene articles]"
4,1000,27-05-2015,"[soda, pickled vegetables]"
...,...,...,...
14958,4999,24-01-2015,"[tropical fruit, berries, other vegetables, yo..."
14959,4999,26-12-2015,"[bottled water, herbs]"
14960,5000,09-03-2014,"[fruit/vegetable juice, onions]"
14961,5000,10-02-2015,"[soda, root vegetables, semi-finished bread]"


In [4]:
encoder = TransactionEncoder()
transactions = encoder.fit(DataFrame["itemDescription"]).transform(DataFrame["itemDescription"])
transactions = transactions.astype('int')
df = pd.DataFrame(transactions, columns=encoder.columns_)
df.head()

,Instant food products,UHT-milk,abrasive cleaner,artif. sweetener,baby cosmetics,bags,baking powder,bathroom cleaner,beef,berries,beverages,bottled beer,bottled water,brandy,brown bread,butter,butter milk,cake bar,candles,candy,canned beer,canned fish,canned fruit,canned vegetables,cat food,cereals,chewing gum,chicken,chocolate,chocolate marshmallow,citrus fruit,cleaner,cling film/bags,cocoa drinks,coffee,condensed milk,cooking chocolate,cookware,cream,cream cheese,curd,curd cheese,decalcifier,dental care,dessert,detergent,dish cleaner,dishes,dog food,domestic eggs,female sanitary products,finished products,fish,flour,flower (seeds),flower soil/fertilizer,frankfurter,frozen chicken,frozen dessert,frozen fish,frozen fruits,frozen meals,frozen potato products,frozen vegetables,fruit/vegetable juice,grapes,hair spray,ham,hamburger meat,hard cheese,herbs,honey,house keeping products,hygiene articles,ice cream,instant coffee,jam,ketchup,kitchen towels,kitchen utensil,light bulbs,liqueur,liquor,liquor (appetizer),liver loaf,long life bakery product,make up remover,male cosmetics,margarine,mayonnaise,meat,meat spreads,misc. beverages,mustard,napkins,newspapers,nut snack,nuts/prunes,oil,onions,organic products,organic sausage,other vegetables,packaged fruit/vegetables,pasta,pastry,pet care,photo/film,pickled vegetables,pip fruit,popcorn,pork,pot plants,potato products,preservation products,processed cheese,prosecco,pudding powder,ready soups,red/blush wine,rice,roll products,rolls/buns,root vegetables,rubbing alcohol,rum,salad dressing,salt,salty snack,sauces,sausage,seasonal products,semi-finished bread,shopping bags,skin care,sliced cheese,snack products,soap,soda,soft cheese,softener,soups,sparkling wine,specialty bar,specialty cheese,specialty chocolate,specialty fat,specialty vegetables,spices,spread cheese,sugar,sweet spreads,syrup,tea,tidbits,toilet cleaner,tropical fruit,turkey,vinegar,waffles,whipped/sour cream,whisky,white bread,white wine,whole milk,yogurt,zwieback
0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,1,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0


In [5]:
print(df.shape)
df.dtypes

(14963, 167)


Instant food products    int32
UHT-milk                 int32
abrasive cleaner         int32
artif. sweetener         int32
baby cosmetics           int32
                         ...  
white bread              int32
white wine               int32
whole milk               int32
yogurt                   int32
zwieback                 int32
Length: 167, dtype: object

Applichiamo l'algoritmo apriori sul df per trovare il frequent itemset con supporto minimo del 2%

In [6]:
df = df.astype("bool")
frequent_itemsets=apriori(df, min_support=0.001)  
frequent_itemsets['itemsets'].shape

(750,)

In [7]:
frequent_itemsets['length'] = frequent_itemsets['itemsets'].apply(lambda x:len(x)) 
frequent_itemsets

,support,itemsets,length
0,0.004010,(0),1
1,0.021386,(1),1
2,0.001470,(2),1
3,0.001938,(3),1
4,0.008087,(6),1
...,...,...,...
745,0.001136,"(122, 164, 130)",3
746,0.001002,"(122, 164, 138)",3
747,0.001337,"(122, 164, 165)",3
748,0.001069,"(130, 164, 138)",3


In [8]:
# sortingthe dataframe
frequent_itemsets= frequent_itemsets.sort_values(by='support', ascending=False) 
# findingtop 5 itemswith minimum supportof 2%
frequent_itemsets[ (frequent_itemsets['length'] == 1) & (frequent_itemsets['support'] >= 0.02) ][0:5]

,support,itemsets,length
146,0.157923,(164),1
90,0.122101,(102),1
109,0.110005,(122),1
123,0.097106,(138),1
147,0.085879,(165),1


In [9]:
frequent_itemsets[(frequent_itemsets['length'] ==2) &(frequent_itemsets['support'] >=0.001)]

,support,itemsets,length
625,0.014837,"(164, 102)",2
677,0.013968,"(122, 164)",2
717,0.011629,"(138, 164)",2
740,0.011161,"(164, 165)",2
609,0.010559,"(122, 102)",2
...,...,...,...
344,0.001002,"(88, 27)",2
201,0.001002,"(27, 11)",2
202,0.001002,"(11, 28)",2
516,0.001002,"(105, 68)",2


In [10]:
rules=association_rules(frequent_itemsets, metric='support', min_threshold=0.001) 
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
0,(164),(102),0.157923,0.122101,0.014837,0.093948,0.769430,-0.004446,0.968928,-0.262461
1,(102),(164),0.122101,0.157923,0.014837,0.121511,0.769430,-0.004446,0.958551,-0.254477
2,(122),(164),0.110005,0.157923,0.013968,0.126974,0.804028,-0.003404,0.964550,-0.214986
3,(164),(122),0.157923,0.110005,0.013968,0.088447,0.804028,-0.003404,0.976350,-0.224474
4,(138),(164),0.097106,0.157923,0.011629,0.119752,0.758296,-0.003707,0.956636,-0.260917
...,...,...,...,...,...,...,...,...,...,...
1233,(28),(11),0.023592,0.045312,0.001002,0.042493,0.937790,-0.000067,0.997056,-0.063618
1234,(105),(68),0.051728,0.021854,0.001002,0.019380,0.886791,-0.000128,0.997477,-0.118652
1235,(68),(105),0.021854,0.051728,0.001002,0.045872,0.886791,-0.000128,0.993862,-0.115446
1236,(14),(15),0.037626,0.035220,0.001002,0.026643,0.756469,-0.000323,0.991188,-0.250666


In [11]:
rules.sort_values(by='confidence', ascending=False)[0:10]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
667,"(130, 165)",(164),0.005748,0.157923,0.001470,0.255814,1.619866,0.000563,1.131541,0.384877
977,"(122, 130)",(164),0.005347,0.157923,0.001136,0.212500,1.345594,0.000292,1.069304,0.258214
1079,"(130, 138)",(164),0.005948,0.157923,0.001069,0.179775,1.138374,0.000130,1.026642,0.122281
527,(132),(164),0.009490,0.157923,0.001671,0.176056,1.114825,0.000172,1.022008,0.103985
827,"(122, 165)",(164),0.007819,0.157923,0.001337,0.170940,1.082428,0.000102,1.015701,0.076751
666,"(130, 164)",(165),0.008955,0.085879,0.001470,0.164179,1.911760,0.000701,1.093681,0.481231
747,(45),(164),0.008621,0.157923,0.001403,0.162791,1.030824,0.000042,1.005814,0.030162
256,(67),(164),0.017109,0.157923,0.002740,0.160156,1.014142,0.000038,1.002659,0.014188
26,(11),(164),0.045312,0.157923,0.007151,0.157817,0.999330,-0.000005,0.999874,-0.000702
1088,(59),(164),0.006817,0.157923,0.001069,0.156863,0.993287,-0.000007,0.998743,-0.006759


In [12]:
rules[(rules['support'] >=0.001) &(rules['lift'] >1.0)]

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
44,(130),(138),0.060349,0.097106,0.005948,0.098560,1.014975,0.000088,1.001613,0.015702
45,(138),(130),0.097106,0.060349,0.005948,0.061253,1.014975,0.000088,1.000963,0.016341
48,(130),(165),0.060349,0.085879,0.005748,0.095238,1.108986,0.000565,1.010345,0.104587
49,(165),(130),0.085879,0.060349,0.005748,0.066926,1.108986,0.000565,1.007049,0.107508
70,(56),(102),0.037760,0.122101,0.005146,0.136283,1.116150,0.000536,1.016420,0.108146
...,...,...,...,...,...,...,...,...,...,...
1223,(123),(73),0.069572,0.013700,0.001002,0.014409,1.051733,0.000049,1.000719,0.052866
1226,(122),(131),0.110005,0.007084,0.001002,0.009113,1.286395,0.000223,1.002048,0.250151
1227,(131),(122),0.007084,0.110005,0.001002,0.141509,1.286395,0.000223,1.036698,0.224222
1228,(88),(27),0.032213,0.027869,0.001002,0.031120,1.116675,0.000105,1.003356,0.107962
